In [ ]:
!pip install pandas

In [ ]:
import pymongo
import pandas as pd
import re
import json
from datetime import datetime

In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017')

In [ ]:
df = pd.read_csv("BankChurners.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
# Splitting the DataFrame into customers_data and transactions_data
customer_columns = ["CLIENTNUM", "Customer_Age", "Gender", "Education_Level", "Marital_Status", "Income_Category", "Card_Category"]
transaction_columns = ["CLIENTNUM", "Total_Trans_Amt", "Total_Revolving_Bal", "Credit_Limit", "Total_Amt_Chng_Q4_Q1", "Total_Ct_Chng_Q4_Q1", "Avg_Utilization_Ratio"]

customers_data = df[customer_columns]
transactions_data = df[transaction_columns]


In [ ]:
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['BankChurners']

# Check if the collection exists and drop it if it does
if "customers" in db.list_collection_names():
    db.customers.drop()
    
# Create collections with schema validation
db.create_collection("customers", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["CLIENTNUM"],
        "properties": {
            "CLIENTNUM": {
                "bsonType": "int",
                "description": "must be a int and is required"
            },
            "Customer_Age": {
                "bsonType": "int",
                "description": "must be an integer"
            },
            "Gender": {
                "bsonType": "string"
            },
            "Education_Level": {
                "bsonType": "string"
            },
            "Marital_Status": {
                "bsonType": "string"
            },
            "Income_Category": {
                "bsonType": "string"
            },
            "Card_Category": {
                "bsonType": "string"
            }
        }
    }
})

# Repeat for transactions collection
if "transactions" in db.list_collection_names():
    db.transactions.drop()

db.create_collection("transactions", validator={
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["CLIENTNUM"],
        "properties": {
            "CLIENTNUM": {
                "bsonType": "int",
                "description": "must be a int and is required"
            },
            "Total_Trans_Amt": {
                "bsonType": "double"
            },
            "Total_Trans_Ct": {
                "bsonType": "double"
            },
            "Total_Amt_Chng_Q4_Q1": {
                "bsonType": "double"
            },
            "Total_Ct_Chng_Q4_Q1": {
                "bsonType": "double"
            },
            "Avg_Utilization_Ratio": {
                "bsonType": "double"
            },
            "Credit_Limit": {
                "bsonType": "double"
            },
            "Total_Revolving_Bal": {
                "bsonType": "double"
            }
        }
    }
})


In [ ]:
# Assuming transactions_data is a DataFrame slice, apply changes using .loc to ensure changes are done in place



In [ ]:
# Ensure the data is in the correct format
customers_dict = customers_data.to_dict('records')
transactions_dict = transactions_data.to_dict('records')

# Attempt to insert data into MongoDB
try:
    db.customers.delete_many({})  # Clear existing data if necessary
    db.transactions.delete_many({})
    
    db.customers.insert_many(customers_dict)
    db.transactions.insert_many(transactions_dict)
    print("Data inserted successfully")
except Exception as e:
    print(f"An error occurred during insertion: {e}")


In [ ]:
# Fetch and print data to verify
print(list(db.customers.find().limit(5)))
print(list(db.transactions.find().limit(5)))
